__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2023-01-10 11:21:19 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2023-01-10 11:21:19 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2023-01-10 11:21:22 | AdsListParser: downloader init
[INFO    ] 2023-01-10 11:21:22 | AvitoParser: init
[INFO    ] 2023-01-10 11:21:26 | AvitoParser: 86 pages for read
[INFO    ] 2023-01-10 11:21:26 | AdsListParser: start read and parse pages...
[INFO    ] 2023-01-10 11:21:30 | AdsListParser: read page 1
[INFO    ] 2023-01-10 11:21:32 | AdsListParser: read page 2
[INFO    ] 2023-01-10 11:21:36 | AdsListParser: read page 3
[INFO    ] 2023-01-10 11:21:40 | AdsListParser: read page 4
[INFO    ] 2023-01-10 11:21:43 | AdsListParser: read page 5
[INFO    ] 2023-01-10 11:21:45 | AdsListParser: read page 6
[INFO    ] 2023-01-10 11:21:51 | AdsListParser: read page 7
[INFO    ] 2023-01-10 11:21:56 | AdsListParser: read page 8
[INFO    ] 2023-01-10 11:21:59 | AdsListParser: read page 9
[INFO    ] 2023-01-10 11:22:05 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3826
3039


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
1899,2297032936,"Объявление «1-к. квартира, 30,4 м², 4/5 эт.» 10 фотографий",4650000,,"Севастополь, улица Хрусталёва, 19","Однокомнатная квартира 30,4 м2, на 4/5 этаже дома. Расположена в г. Севастополь, Ленинский район, ул. Хрусталева, д. 19. \n\nПозвоните нам, чтобы узнать все детали! \n\nТехнические характеристики: \n— Адрес: г. Севастополь, Ленинский район, ул. Хрусталева, д. 1",46,2023-01-10 11:25:50.588268,sevastopol
933,2505351992,"Объявление «2-к. квартира, 59 м², 2/9 эт.» 4 фотографии",7080000,ЖК «Доброгород»,"ул. Токарева, 7 этап, блок-секция 2","Крупнейший в Севастополе семейный микрорайон «Доброгород»! \n\nПобедитель всероссийской премии «Топ ЖК 2021» в номинации «Лучший жилой комплекс». \n\nИпотека с господдержкой, рассрочка от Застройщика. \n\n«Доброгород» по праву считается лучшим жилым кварталом Се",20,2023-01-10 11:25:50.588268,sevastopol


In [11]:
# !pip install dash_table